<a href="https://colab.research.google.com/github/SAMARTH-1301/SAMPLING_SAMARTH_102203487/blob/main/SAMPLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

data = pd.read_csv('/content/Creditcard_data.csv')

majority_class = data[data.Class == 0]
minority_class = data[data.Class == 1]

minority_upsampled = resample(minority_class,
                              replace=True,
                              n_samples=len(majority_class),
                              random_state=42)

balanced_data = pd.concat([majority_class, minority_upsampled])

sample_size = int(0.5 * len(balanced_data))

sample1 = balanced_data.sample(n=sample_size, random_state=1)

_, sample2 = train_test_split(balanced_data, test_size=sample_size, stratify=balanced_data['Class'], random_state=2)

step = len(balanced_data) // sample_size
sample3_indices = np.arange(0, len(balanced_data), step)[:sample_size]
sample3 = balanced_data.iloc[sample3_indices]

clusters = np.array_split(balanced_data.sample(frac=1, random_state=4), 10)
sample4 = pd.concat(clusters[:len(clusters) // 2])

oversample = resample(minority_upsampled, replace=True, n_samples=sample_size // 2, random_state=5)
sample5 = pd.concat([oversample, majority_class.sample(n=sample_size // 2, random_state=5)])

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=0),
    "Random Forest": RandomForestClassifier(random_state=0),
    "SVM": SVC(random_state=0),
    "KNN": KNeighborsClassifier(),
    "Neural Network": MLPClassifier(max_iter=1000, random_state=0)
}

model_accuracies = {model_name: [] for model_name in models.keys()}

samples = [sample1, sample2, sample3, sample4, sample5]
for model_name, model in models.items():
    for sample in samples:
        X = sample.drop(columns=['Class', 'Time'])
        y = sample['Class']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        model_accuracies[model_name].append(accuracy)

best_sampling_methods = {}
for model_name, accuracies in model_accuracies.items():
    best_sampling_index = np.argmax(accuracies)
    best_sampling_methods[model_name] = f"Sampling{best_sampling_index + 1}"

results = pd.DataFrame.from_dict(model_accuracies, orient='index')
results.columns = [f"Sampling{i+1}" for i in range(len(samples))]
results["Best Sampling Method"] = list(best_sampling_methods.values())

print(results.to_string())




/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


                     Sampling1  Sampling2  Sampling3  Sampling4  Sampling5 Best Sampling Method
Logistic Regression   0.921397   0.899563   0.895197   0.917391   0.903930            Sampling1
Random Forest         0.995633   1.000000   0.995633   1.000000   1.000000            Sampling2
SVM                   0.681223   0.707424   0.742358   0.756522   0.676856            Sampling4
KNN                   0.947598   0.965066   0.943231   0.965217   0.943231            Sampling4
Neural Network        0.969432   0.982533   0.991266   0.991304   1.000000            Sampling5
